Melika Ahmadi Ranjbar 97521036

---
Homework 9


#Question 2

In [ ]:
import numpy as np
import cv2

# Read Image In Gray 
Image = cv2.imread("img1.jpg", 0)

# Apply Various Filters
KernelA = np.array([[1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1]])

KernelA = KernelA / sum(KernelA)
ResultA = cv2.filter2D(Image, -1, KernelA)
cv2.imwrite('ResultA.jpg', ResultA)

KernelAp = np.array([[1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1]])

KernelAp = KernelAp / sum(KernelAp)
ResultAp = cv2.filter2D(Image, -1, KernelAp)
cv2.imwrite('ResultAp.jpg', ResultAp)

KernelB= np.array([[-1, -1, -1],
                    [-1, 8, -1],
                    [-1, -1, -1]])

ResultB = cv2.filter2D(Image, -1, KernelB)
cv2.imwrite('ResultB.jpg', ResultB)

KernelC = np.array([[-1, -2, -1],
                    [0, 0, 0],
                    [1, 2, 1]])

ResultC = cv2.filter2D(Image, -1, KernelC)
cv2.imwrite('ResultC.jpg', ResultC)

KernelD = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]])

ResultD = cv2.filter2D(Image, -1, KernelD)
cv2.imwrite('ResultD.jpg', ResultD)

True

#Question 3


In [3]:
import numpy as np
import keras_tuner as kt
from tensorflow import keras
from keras.datasets import cifar10
from tensorflow.keras import layers

# !pip install -q -U keras-tuner

# Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

Classes = 10
y_train = keras.utils.to_categorical(y_train, Classes)
y_test = keras.utils.to_categorical(y_test, Classes)

In [4]:
def BuildModel(Hp):

    Model = keras.Sequential()
    # Convolution
    Model.add(
            layers.Conv2D(
                filters=Hp.Int(f"filters", min_value=32, max_value=256, step=32),
                kernel_size=(3, 3),
                activation="relu",
                input_shape=(32, 32, 3),
                padding="same"
            )
        )
    
    for i in range(Hp.Int("num_layers", 0, 4)):
        Model.add(
            layers.Conv2D(
                # Tune Number Of Filters Separately.
                filters=Hp.Int(f"filters_{i}", min_value=32, max_value=256, step=32),
                kernel_size=(3, 3),
                activation="relu",
                padding="same"
            )
        )
        if Hp.Boolean("dropout"):
            Model.add(layers.Dropout(rate=0.1))

    
    Model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    Model.add(layers.Flatten())
    # Dense
    for i in range(Hp.Int("num_layers", 0, 4)):
        Model.add(
            layers.Dense(
                # Tune Number Of Units Separately.
                units=Hp.Int(f"units_{i}", min_value=32, max_value=256, step=32),
                activation=Hp.Choice("activation", ["relu", "sigmoid"]),
            )
        )

    if Hp.Boolean("dropout"):
        Model.add(layers.Dropout(rate=0.1))

    # Final Layer
    Model.add(layers.Dense(10, activation="softmax"))

    # Compile
    LearningRate = Hp.Float("lr", min_value=1e-4, max_value=1e-3, sampling="log")
    Optimizer = Hp.Choice("optimizer", values=['adam', 'SGD'])

    if Optimizer == 'adam':
        Optimizer= keras.optimizers.Adam(learning_rate=LearningRate)
    elif Optimizer == 'SGD':
        Optimizer= keras.optimizers.SGD(learning_rate=LearningRate)

    Model.compile(optimizer=Optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return Model

BuildModel(kt.HyperParameters())

In [5]:
# Other Arguments Are Set To Default
Tuner = kt.BayesianOptimization(
    hypermodel=BuildModel,
    objective="val_accuracy",
    max_trials=3
)
Tuner.search_space_summary()

Search space summary
Default search space size: 5
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 4, 'step': 1, 'sampling': None}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.001, 'step': None, 'sampling': 'log'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'SGD'], 'ordered': False}


In [6]:
Tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Trial 3 Complete [00h 05m 23s]
val_accuracy: 0.6704000234603882

Best val_accuracy So Far: 0.6704000234603882
Total elapsed time: 00h 19m 54s
INFO:tensorflow:Oracle triggered exit


In [7]:
# Get The Optimal Hyperparameters
BestHp = Tuner.get_best_hyperparameters(num_trials=1)[0]

Models = Tuner.get_best_models(num_models=2)
BestModel = Models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
BestModel.summary()

Tuner.results_summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 average_pooling2d (AverageP  (None, 16, 16, 32)       0         
 ooling2D)                                                       
                                                        

In [9]:
# Retrain
Model = Tuner.hypermodel.build(BestHp)
History = Model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1563/1563 [==============================] - 27s 17ms/step - loss: 1.7292 - accuracy: 0.3570 - val_loss: 1.3921 - val_accuracy: 0.5022
Epoch 2/5
1563/1563 [==============================] - 26s 17ms/step - loss: 1.1848 - accuracy: 0.5767 - val_loss: 1.0685 - val_accuracy: 0.6244
Epoch 3/5
1563/1563 [==============================] - 27s 17ms/step - loss: 0.9242 - accuracy: 0.6762 - val_loss: 1.0607 - val_accuracy: 0.6361
Epoch 4/5
1563/1563 [==============================] - 26s 17ms/step - loss: 0.7241 - accuracy: 0.7493 - val_loss: 1.0635 - val_accuracy: 0.6540
Epoch 5/5
1563/1563 [==============================] - 27s 17ms/step - loss: 0.5394 - accuracy: 0.8129 - val_loss: 1.0686 - val_accuracy: 0.6648
